In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt
import numpy as np
# import os
# import pandas as pd

2023-06-09 15:54:22.587654: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Dataset preparation

### Set parameters

In [2]:
batch_size = 32
img_height = 184
img_width = 216
img_size = (img_height, img_width)
img_shape = img_size + (3,)
ds_path = "/drive0-storage/Gracia/dataset_1"

### Load dataset

In [3]:
from tensorflow.keras.utils import image_dataset_from_directory

with tf.device("CPU"):
    train_ds = image_dataset_from_directory(ds_path,
                                        validation_split = 0.2,
                                        subset = "training",
                                        seed = 123,
                                        image_size = img_size,
                                        batch_size = batch_size)
    
    val_ds = image_dataset_from_directory(ds_path,
                                      validation_split = 0.2,
                                      subset = "validation",
                                      seed = 123,
                                      image_size = img_size,
                                      batch_size = batch_size)

2023-06-09 15:54:24.574646: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 15:54:25.121855: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1637] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7910 MB memory:  -> device: 0, name: A100-SXM4-40GB MIG 2g.10gb, pci bus id: 0000:90:00.0, compute capability: 8.0


Found 21408 files belonging to 3 classes.
Using 17127 files for training.
Found 21408 files belonging to 3 classes.
Using 4281 files for validation.


In [4]:
#os.rmdir("/drive0-storage/Gracia/dataset/.ipynb_checkpoints")

In [5]:
class_names = train_ds.class_names
num_classes = len(class_names)

print(class_names)

['bipolar_disorder', 'healthy_controls', 'schizophrenia']


### Test set

In [6]:
with tf.device("CPU"):
    val_batches = tf.data.experimental.cardinality(val_ds)
    test_dataset = val_ds.take(val_batches // 2)
    validation_dataset = val_ds.skip(val_batches // 2)

In [7]:
# Buffered prefetching
AUTOTUNE = tf.data.AUTOTUNE

with tf.device("CPU"):
    train_dataset = train_ds.prefetch(buffer_size = AUTOTUNE)
    validation_dataset = validation_dataset.prefetch(buffer_size = AUTOTUNE)
    test_dataset = test_dataset.prefetch(buffer_size = AUTOTUNE)

In [8]:
for images in train_ds.take(1):
    img = images[0]
    print(img.shape)  
    print(img.dtype)  
    print(np.min(img),np.max(img))  

(32, 184, 216, 3)
<dtype: 'float32'>
0.0 248.0


In [9]:
len(validation_dataset)

67

## Model building

In [10]:
def build_vgg16():
    model = Sequential()
    
    # Block 1
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=img_shape))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 4
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 5
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Flatten the output and create fully connected layers
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))  
    
    return model

In [20]:
with tf.device("CPU"):
    vgg16 = build_vgg16()

    # add rescale layer, chain model
    rescale = tf.keras.layers.Rescaling(1./255)
    inputs = tf.keras.Input(shape = img_shape)
    x = rescale(inputs)
    outputs = vgg16(x)
    model = tf.keras.Model(inputs,outputs) 
    
    # compile model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-5),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
                  metrics=['accuracy'])

## Model training

In [ ]:
# Callback function
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.95):
            print("\nReached 95% accuracy so cancelling training!")
            self.model.stop_training = True
            
callbacks = myCallback()

In [12]:
## batch 16, optimizer adam, lr 1e-5
EPOCH = 8

history_1 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH,
            batch_size = 16)
#             callbacks=[callbacks])

Epoch 1/8


2023-06-09 15:54:31.209406: I tensorflow/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-06-09 15:54:32.497949: I tensorflow/stream_executor/cuda/cuda_blas.cc:1633] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


536/536 [==============================] - 165s 300ms/step - loss: 1.0010 - accuracy: 0.5582 - val_loss: 1.0056 - val_accuracy: 0.5433
Epoch 2/8
536/536 [==============================] - 159s 296ms/step - loss: 0.9966 - accuracy: 0.5583 - val_loss: 1.0051 - val_accuracy: 0.5438
Epoch 3/8
536/536 [==============================] - 158s 295ms/step - loss: 0.9939 - accuracy: 0.5583 - val_loss: 1.0018 - val_accuracy: 0.5461
Epoch 4/8
536/536 [==============================] - 159s 296ms/step - loss: 0.9891 - accuracy: 0.5583 - val_loss: 0.9937 - val_accuracy: 0.5414
Epoch 5/8
536/536 [==============================] - 158s 295ms/step - loss: 0.9614 - accuracy: 0.5655 - val_loss: 0.9596 - val_accuracy: 0.5676
Epoch 6/8
536/536 [==============================] - 159s 296ms/step - loss: 0.9051 - accuracy: 0.5959 - val_loss: 0.8993 - val_accuracy: 0.6013
Epoch 7/8
536/536 [==============================] - 158s 295ms/step - loss: 0.8734 - accuracy: 0.6096 - val_loss: 0.9259 - val_accuracy: 0.

In [15]:
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

67/67 [==============================] - 5s 79ms/step - loss: 0.5382 - accuracy: 0.7850


{'loss': 0.5381730794906616, 'accuracy': 0.784981369972229}

In [18]:
## batch 32, optimizer adam, lr 1e-5
EPOCH = 8

history_2 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH,
            batch_size = 32)
#             callbacks=[callbacks])

Epoch 1/8
536/536 [==============================] - 161s 299ms/step - loss: 0.9998 - accuracy: 0.5581 - val_loss: 1.0117 - val_accuracy: 0.5363
Epoch 2/8
536/536 [==============================] - 160s 299ms/step - loss: 0.9959 - accuracy: 0.5583 - val_loss: 1.0079 - val_accuracy: 0.5405
Epoch 3/8
536/536 [==============================] - 161s 300ms/step - loss: 0.9939 - accuracy: 0.5583 - val_loss: 1.0018 - val_accuracy: 0.5438
Epoch 4/8
536/536 [==============================] - 161s 300ms/step - loss: 0.9877 - accuracy: 0.5583 - val_loss: 0.9935 - val_accuracy: 0.5475
Epoch 5/8
536/536 [==============================] - 160s 299ms/step - loss: 0.9528 - accuracy: 0.5697 - val_loss: 0.9408 - val_accuracy: 0.5891
Epoch 6/8
536/536 [==============================] - 160s 299ms/step - loss: 0.9074 - accuracy: 0.5932 - val_loss: 0.9027 - val_accuracy: 0.5873
Epoch 7/8
536/536 [==============================] - 160s 299ms/step - loss: 0.8816 - accuracy: 0.6042 - val_loss: 0.8829 - val_ac

In [19]:
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

67/67 [==============================] - 4s 64ms/step - loss: 0.8444 - accuracy: 0.6236


{'loss': 0.8443752527236938, 'accuracy': 0.6236007213592529}

In [ ]:
## batch 64, optimizer adam, lr 1e-5
EPOCH = 8

history_3 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH,
            batch_size = 64)
#             callbacks=[callbacks])

Epoch 1/8
 44/536 [=>............................] - ETA: 2:21 - loss: 1.0287 - accuracy: 0.5547

In [ ]:
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

In [ ]:
## batch 16, optimizer adam, lr 1e-4
EPOCH = 8

history_4 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH,
            batch_size = 16)
#             callbacks=[callbacks])

In [ ]:
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

In [ ]:
## batch 32, optimizer adam, lr 1e-4
EPOCH = 8

history_5 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH,
            batch_size = 32)
#             callbacks=[callbacks])

In [ ]:
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

In [ ]:
## batch 64, optimizer adam, lr 1e-4
EPOCH = 8

history_6 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH,
            batch_size = 64)
#             callbacks=[callbacks])

In [ ]:
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

In [ ]:
## batch 16, optimizer adam, lr 1e-3
EPOCH = 8

history_7 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH,
            batch_size = 16)
#             callbacks=[callbacks])

In [ ]:
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

In [ ]:
## batch 32, optimizer adam, lr 1e-3
EPOCH = 8

history_8 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH,
            batch_size = 32)
#             callbacks=[callbacks])

In [ ]:
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

In [ ]:
## batch 64, optimizer adam, lr 1e-3
EPOCH = 8

history_9 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH,
            batch_size = 64)
#             callbacks=[callbacks])

In [ ]:
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

In [ ]:
## batch 16, optimizer sgd, lr 1e-5
EPOCH = 8

history_10 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH)
#             callbacks=[callbacks])

In [ ]:
## batch 32, optimizer sgd, lr 1e-5
EPOCH = 8

history_11 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH)
#             callbacks=[callbacks])

In [ ]:
## batch 64, optimizer sgd, lr 1e-5
EPOCH = 8

history_12 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH)
#             callbacks=[callbacks])

In [ ]:
## batch 16, optimizer sgd, lr 1e-4
EPOCH = 8

history_13 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH)
#             callbacks=[callbacks])

In [ ]:
## batch 32, optimizer sgd, lr 1e-4
EPOCH = 8

history_14 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH)
#             callbacks=[callbacks])

In [ ]:
## batch 64, optimizer sgd, lr 1e-4
EPOCH = 8

history_15 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH)
#             callbacks=[callbacks])

In [ ]:
## batch 16, optimizer sgd, lr 1e-3
EPOCH = 8

history_16 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH)
#             callbacks=[callbacks])

In [ ]:
## batch 32, optimizer sgd, lr 1e-3
EPOCH = 8

history_17 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH)
#             callbacks=[callbacks])

In [ ]:
## batch 64, optimizer sgd, lr 1e-3
EPOCH = 8

history_18 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH)
#             callbacks=[callbacks])

### Training evaluation

In [ ]:
plt.figure(figsize=(8, 8))

# training and validation accuracy
acc = history_1.history['accuracy']
val_acc = history_1.history['val_accuracy']

plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.ylim([min(plt.ylim()),1])
plt.title('Accuracy', size=15, fontweight='bold')

# training and validation loss
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch')
plt.ylim([0,1.0])
plt.title('Loss', size=15, fontweight='bold')


plt.subplots_adjust(hspace=0.3)
plt.show()